## Simulate a collection of Molecules 
* single mol file 
* added to a simulation'
* Avoid periodic boundary conditions such that no molecule is outside of the box

In [1]:
from sys import stdout

# OpenMM imports
import openmm.app as app
import openmm as mm
import openmm.unit as unit
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

# OpenFF-toolkit imports
from openff.toolkit import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm



In [2]:
mixture_path = "added_to_minimized_sample_single_mol.sdf"
mixture = Molecule.from_file(mixture_path)

In [3]:
# Create the SMIRNOFF template generator with the default force field
smirnoff = SMIRNOFFTemplateGenerator(molecules=mixture)

# We can check which version of the force field is being used
print(smirnoff.smirnoff_filename)

# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P
ff = app.ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml')

# Add in the SMIRNOFF template generator
ff.registerTemplateGenerator(smirnoff.generator)

/home/julian/anaconda3/envs/mixturesimlation/lib/python3.11/site-packages/openforcefields/offxml/openff-2.1.0.offxml


In [5]:
from openff.toolkit.topology import Topology
from openff.units import unit
# Make an OpenFF Topology of the ligand
ligand_off_topology = offTopology.from_molecules(molecules=mixture)


# Define a box with edge lengths (nanometers)
box_vectors = [2.0, 2.0, 2.0] * unit.nanometer
ligand_off_topology.box_vectors = box_vectors
# Convert it to an OpenMM Topology
ligand_omm_topology = ligand_off_topology.to_openmm()

# Get the positions of the ligand
 
ligand_positions = offquantity_to_openmm(mixture.conformers[0])

# Add the ligand to the Modeller
modeller = app.Modeller(ligand_omm_topology, ligand_positions)

In [6]:
import openmm.unit as unit
# Create the system, define the integrator, and create the simulation
system = ff.createSystem(modeller.topology, nonbondedMethod=app.PME, constraints=app.HBonds)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(modeller.topology, system, integrator)

# set the positions
simulation.context.setPositions(modeller.positions)

print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=10000)

simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

simulation.reporters.append(app.PDBReporter('traj.pdb', 10))
simulation.reporters.append(app.DCDReporter('traj.dcd', 10))


simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, speed=True))

print("Running simulation...")
simulation.step(10000)

/home/julian/anaconda3/envs/mixturesimlation/lib/python3.11/site-packages/openff/interchange/smirnoff/_nonbonded.py:472: UserWarning: Warning! Partial charge method 'am1bcc' is not designed for use on large (i.e. > 150 atoms) molecules and may crash or take hours to run on this molecule (found 702 atoms). For more, see https://docs.openforcefield.org/projects/toolkit/en/stable/faq.html#parameterizing-my-system-which-contains-a-large-molecule-is-taking-forever-whats-wrong
  molecule.assign_partial_charges(method)
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
/home/julian/anaconda3/envs/mixturesimlation/bin/wrapped_progs/antechamber: Fatal Error!
Unable to find sqm charges in file (sqm.out).
Verify the filename and the file contents.


ValueError: No registered toolkits can provide the capability "assign_partial_charges" for args "()" and kwargs "{'molecule': Molecule with name '' and SMILES '[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H])([H])[C]([H])([H])[H].[H][O][C]([H

In [10]:
command = ['antechamber', '-i', 'molecule.sdf', '-fi', 'sdf', '-o', 'charged.mol2', '-fo', 'mol2', '-pf', 'yes', '-dr', 'n', '-c', 'bcc', '-nc', '0.0']

In [12]:
" ".join(command)

'antechamber -i molecule.sdf -fi sdf -o charged.mol2 -fo mol2 -pf yes -dr n -c bcc -nc 0.0'